In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import ast

In [ ]:
import os

# Specify the directory path
directory_path = '/content/drive/MyDrive/embeddings'

# List all files in the directory
all_files = os.listdir(directory_path)

# Filter out only files (exclude directories)
files = [f for f in all_files if os.path.isfile(os.path.join(directory_path, f))]

# Print the list of files
print(len(files))

322


In [ ]:
def average_embeddings(embeddings):
    embeddings_list = [ast.literal_eval(emb) for emb in embeddings]
    stacked_embeddings = np.stack(embeddings_list)
    return np.mean(stacked_embeddings, axis=0)

In [ ]:
# Group by 'date' and compute the average of embeddings
df_avg = df.groupby(['publishedDate', 'symbol'], as_index=False).agg({'Embeddings': average_embeddings})
df_avg = df_avg[df_avg['publishedDate'] > '2015-12-31']


    publishedDate symbol                                         Embeddings
105    2018-10-11   GOOG  [0.25184798, -0.51663476, -0.1723049, 0.422058...
106    2018-10-12   GOOG  [0.101739627, -0.67832665, 0.22173161000000002...
107    2018-10-15   GOOG  [0.29478607, -0.446698, 0.40445566, 0.4483078,...
108    2018-11-02   GOOG  [0.26638764, -0.61933434, 0.18396121, 0.376758...
109    2018-11-16   GOOG  [0.317116, -0.67017, -0.014671727, 0.5381052, ...
110    2018-11-19   GOOG  [-0.5168457, -0.94873047, 0.5717076, 0.1305018...
111    2018-12-07   GOOG  [0.270906415, -0.72638493, 0.5840465, 0.454084...
112    2018-12-09   GOOG  [0.29178512, -0.5142769, 0.27838108, 0.5905989...
113    2018-12-12   GOOG  [-0.37451643, -0.064120844, 0.8477126, 0.04074...
114    2019-01-10   GOOG  [-0.059145864, -0.95953774, 0.50088704, 0.2747...
115    2019-01-21   GOOG  [-0.19409864, -0.36720276, 1.1781006, -0.19193...
116    2019-01-22   GOOG  [-0.10766912, -0.722611, 0.36804143, 0.3105329...
117    2019-

In [ ]:
final_df = pd.DataFrame(columns=['publishedDate', 'embedding', 'symbol'])

# Directory containing the files
directory_path = '/content/drive/MyDrive/embeddings'  # Replace with your directory path

# Step 2: Read files from the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):  # Assuming the files are CSV; modify if necessary
        file_path = os.path.join(directory_path, filename)

        # Step 3: Read each file into a DataFrame
        df = pd.read_csv(file_path)

        # Step 3.1: Drop any columns that are not 'published date' or 'embedding'
        df = df[['publishedDate', 'Embeddings']]  # Keep only the relevant columns

        # Step 3.2: Remove the time component from 'published date' and retain only the year
        df['publishedDate'] = df['publishedDate'].str.split(' ').str[0]  # Keep only the date part
        df['date'] = pd.to_datetime(df['publishedDate']).dt.year  # Extract the year as integer

        # Step 3.3: Convert the 'embedding' column to a list of NumPy arrays
        #df['Embeddings'] = df['Embeddings'].apply(lambda x: np.array(eval(x)))

        # Step 4: Compute the average of embeddings for each unique date
        df_avg = df.groupby('publishedDate', as_index=False).agg({'Embeddings': average_embeddings})
        df_avg = df_avg[df_avg['publishedDate'] > '2015-12-31']

        # Step 5: Add the 'ticker' column to the averaged DataFrame
        ticker_name = filename.split('.')[0]  # Assuming the ticker name is the filename without extension
        df_avg['symbol'] = ticker_name

        # Step 6: Append the resulting DataFrame to the empty DataFrame
        final_df = pd.concat([final_df, df_avg], ignore_index=True)

# Display the final DataFrame
print(final_df)

<ipython-input-15-6ed81b639b25>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['publishedDate'] = df['publishedDate'].str.split(' ').str[0]  # Keep only the date part
<ipython-input-15-6ed81b639b25>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['publishedDate']).dt.year  # Extract the year as integer
<ipython-input-15-6ed81b639b25>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

       publishedDate embedding symbol  \
0         2016-02-10       NaN    REG   
1         2016-05-03       NaN    REG   
2         2016-08-02       NaN    REG   
3         2016-11-01       NaN    REG   
4         2016-11-14       NaN    REG   
...              ...       ...    ...   
246191    2024-07-17       NaN   GOOG   
246192    2024-07-18       NaN   GOOG   
246193    2024-07-19       NaN   GOOG   
246194    2024-07-20       NaN   GOOG   
246195    2024-07-21       NaN   GOOG   

                                               Embeddings  
0       [0.29509735, -0.4161682, 0.88429767, 0.5892944...  
1       [0.2488183, -0.4543457, 0.32148403, 0.26011148...  
2       [0.56136066, -0.3433126, 0.36288112, 0.2157796...  
3       [0.37261283, -0.32358128, 0.66452366, 0.125385...  
4       [-0.18272829, -0.65021515, 0.17470932, 0.43341...  
...                                                   ...  
246191  [-0.2906501932941176, -0.2779231380411765, 0.3...  
246192  [-0.272093228933333

In [ ]:
print(final_df)

       publishedDate embedding symbol  \
0         2016-02-10       NaN    REG   
1         2016-05-03       NaN    REG   
2         2016-08-02       NaN    REG   
3         2016-11-01       NaN    REG   
4         2016-11-14       NaN    REG   
...              ...       ...    ...   
246191    2024-07-17       NaN   GOOG   
246192    2024-07-18       NaN   GOOG   
246193    2024-07-19       NaN   GOOG   
246194    2024-07-20       NaN   GOOG   
246195    2024-07-21       NaN   GOOG   

                                               Embeddings  
0       [0.29509735, -0.4161682, 0.88429767, 0.5892944...  
1       [0.2488183, -0.4543457, 0.32148403, 0.26011148...  
2       [0.56136066, -0.3433126, 0.36288112, 0.2157796...  
3       [0.37261283, -0.32358128, 0.66452366, 0.125385...  
4       [-0.18272829, -0.65021515, 0.17470932, 0.43341...  
...                                                   ...  
246191  [-0.2906501932941176, -0.2779231380411765, 0.3...  
246192  [-0.272093228933333

In [ ]:
final_df.to_csv('/content/drive/MyDrive/final_df_embeddings.csv')

NameError: name 'final_df' is not defined

In [ ]:
final_df = pd.read_csv('/content/drive/MyDrive/final_df_embeddings.csv')

In [ ]:
print(final_df)

        Unnamed: 0 publishedDate  embedding symbol  \
0                0    2016-02-10        NaN    REG   
1                1    2016-05-03        NaN    REG   
2                2    2016-08-02        NaN    REG   
3                3    2016-11-01        NaN    REG   
4                4    2016-11-14        NaN    REG   
...            ...           ...        ...    ...   
246191      246191    2024-07-17        NaN   GOOG   
246192      246192    2024-07-18        NaN   GOOG   
246193      246193    2024-07-19        NaN   GOOG   
246194      246194    2024-07-20        NaN   GOOG   
246195      246195    2024-07-21        NaN   GOOG   

                                               Embeddings  
0       [ 2.95097350e-01 -4.16168200e-01  8.84297670e-...  
1       [ 2.48818300e-01 -4.54345700e-01  3.21484030e-...  
2       [ 0.56136066 -0.3433126   0.36288112  0.215779...  
3       [ 3.72612830e-01 -3.23581280e-01  6.64523660e-...  
4       [-0.18272829 -0.65021515  0.17470932  0.433

In [ ]:
x = final_df.iloc[0]
print(len(x['Embeddings']))

12480


In [ ]:
total_df = pd.read_csv('/content/drive/MyDrive/total_df.csv')

In [ ]:
print(total_df)
total_df.to_csv('/content/drive/MyDrive/total_df.csv')

        Unnamed: 0        date        open        high         low  \
0                0  2016-01-04  136.030029  138.060457  135.447208   
1                1  2016-01-05  137.703265  139.254290  135.099411   
2                2  2016-01-06  134.855021  136.180436  134.187602   
3                3  2016-01-07  132.542533  134.384975  131.818715   
4                4  2016-01-08  133.482544  133.604751  130.267685   
...            ...         ...         ...         ...         ...   
421024      421024  2024-03-22  123.862256  124.140574  122.947778   
421025      421025  2024-03-25  123.136631  123.484537  122.331494   
421026      421026  2024-03-26  122.301669  123.116748  121.655568   
421027      421027  2024-03-27  123.226092  123.921889  122.470651   
421028      421028  2024-03-28  124.249916  124.746915  122.808620   

             close   volume  dividends  stock splits   day   tic  adj close  \
0       137.759659  1461700        0.0           0.0     0    CI        NaN   
1

In [ ]:
print(total_df.iloc[-1])

date                      2019-12-31
open                       117.31075
high                      117.875899
low                       116.420221
close                     117.781708
volume                        972500
dividends                        0.0
stock splits                     0.0
day                             1005
tic                              KMB
adj close                        NaN
close_pct_change            0.166415
upperband                 118.423845
middleband                 116.95715
lowerband                 115.490454
moving_averages_list       98.171395
year                            2019
cashflow                1527000000.0
Name: 423353, dtype: object


In [ ]:
for tico in y:
  df_slice = total_df[total_df['tic'] == tico]
  #print(tico)
  x = df_slice.iloc[-1]
  #print(type(x['year']))
  if( x['year'] < 2018 ):
     print(tico)
     #print(x)

In [ ]:
y = list(set(total_df['tic']))

In [ ]:
total_df = total_df[total_df['tic'] != "KMB"]

In [ ]:
df_slice = total_df[total_df['tic'] == "KMB"]
  #print(tico)
x = total_df.iloc[-1]
print(x)

date                    2024-03-28
open                    124.249916
high                    124.746915
low                      122.80862
close                   123.146576
volume                     2280800
dividends                      0.0
stock splits                   0.0
day                           2072
tic                           PCAR
adj close                      NaN
close_pct_change          1.887817
upperband               126.224695
middleband              117.692002
lowerband               109.159308
moving_averages_list      47.29495
year                          2024
cashflow                       NaN
Name: 421028, dtype: object


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
final_df = pd.read_csv('/content/drive/MyDrive/final_df_embeddings.csv')

In [ ]:
print(len(set(final_df['symbol'])))

322


In [ ]:
total_df = pd.read_csv('/content/drive/MyDrive/total_df.csv')

In [ ]:
print(total_df)

        Unnamed: 0.1  Unnamed: 0        date        open        high  \
0                  0           0  2016-01-04  136.030029  138.060457   
1                  1           1  2016-01-05  137.703265  139.254290   
2                  2           2  2016-01-06  134.855021  136.180436   
3                  3           3  2016-01-07  132.542533  134.384975   
4                  4           4  2016-01-08  133.482544  133.604751   
...              ...         ...         ...         ...         ...   
421024        421024      421024  2024-03-22  123.862256  124.140574   
421025        421025      421025  2024-03-25  123.136631  123.484537   
421026        421026      421026  2024-03-26  122.301669  123.116748   
421027        421027      421027  2024-03-27  123.226092  123.921889   
421028        421028      421028  2024-03-28  124.249916  124.746915   

               low       close   volume  dividends  stock splits   day   tic  \
0       135.447208  137.759659  1461700        0.0     

In [ ]:
import numpy as np
zero_vector = np.zeros(10)
total_df['Embeddings'] = [zero_vector for _ in range(len(total_df))]

In [ ]:
print(total_df)

        Unnamed: 0.1  Unnamed: 0        date        open        high  \
0                  0           0  2016-01-04  136.030029  138.060457   
1                  1           1  2016-01-05  137.703265  139.254290   
2                  2           2  2016-01-06  134.855021  136.180436   
3                  3           3  2016-01-07  132.542533  134.384975   
4                  4           4  2016-01-08  133.482544  133.604751   
...              ...         ...         ...         ...         ...   
421024        421024      421024  2024-03-22  123.862256  124.140574   
421025        421025      421025  2024-03-25  123.136631  123.484537   
421026        421026      421026  2024-03-26  122.301669  123.116748   
421027        421027      421027  2024-03-27  123.226092  123.921889   
421028        421028      421028  2024-03-28  124.249916  124.746915   

               low       close   volume  dividends  stock splits  ...   tic  \
0       135.447208  137.759659  1461700        0.0      

In [ ]:
filtered_df = total_df[total_df['tic'].isin(intersection_list)]

In [ ]:
total_df_stocks = set(total_df['tic'])

In [ ]:
final_df_stocks = set(final_df['symbol'])

In [ ]:
intersection = final_df_stocks.intersection(total_df_stocks)

In [ ]:
intersection_list = list(intersection)

In [ ]:
print(len(intersection_list))

204


In [ ]:
compressed_news_embeddings = pd.DataFrame()

In [ ]:

for sym in set(intersection_list):

      sample_df = final_df[final_df['symbol'] == sym]
      print(sample_df)
      embeddings = np.array([np.fromstring(x[1:-1], dtype=float, sep=' ') for x in sample_df['Embeddings']])

      # Apply PCA to reduce dimensionality
      pca = PCA(n_components=10)  # Reduce to 10 dimensions, for example
      compressed_embeddings = pca.fit_transform(embeddings)

      # Add the compressed embeddings back into the DataFrame as a new column
      sample_df['Compressed_Embeddings'] = compressed_embeddings.tolist()
      sample_df['tic'] = final_df['symbol']
      compressed_news_embeddings = pd.concat([ compressed_news_embeddings,sample_df], ignore_index=True)


        Unnamed: 0 publishedDate  embedding symbol  \
179378      179378    2016-10-17        NaN    STZ   
179379      179379    2019-09-25        NaN    STZ   
179380      179380    2019-10-04        NaN    STZ   
179381      179381    2019-12-03        NaN    STZ   
179382      179382    2019-12-09        NaN    STZ   
...            ...           ...        ...    ...   
179839      179839    2024-07-03        NaN    STZ   
179840      179840    2024-07-04        NaN    STZ   
179841      179841    2024-07-05        NaN    STZ   
179842      179842    2024-07-08        NaN    STZ   
179843      179843    2024-07-11        NaN    STZ   

                                               Embeddings  
179378  [-0.10805868 -0.26979524  0.11729533  0.296641...  
179379  [-2.30529790e-01  1.52359010e-01  7.03643800e-...  
179380  [ 0.2009377   0.15128723  0.10298969  0.277860...  
179381  [-2.45274140e-02 -3.81090430e-01  1.14055530e+...  
179382  [ 8.80992000e-02  2.20116620e-01  4.7325373

<ipython-input-19-d1fbd9a6ecb2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['Compressed_Embeddings'] = compressed_embeddings.tolist()
<ipython-input-19-d1fbd9a6ecb2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['tic'] = final_df['symbol']


        Unnamed: 0 publishedDate  embedding symbol  \
196714      196714    2017-02-23        NaN    BSX   
196715      196715    2017-02-24        NaN    BSX   
196716      196716    2019-04-30        NaN    BSX   
196717      196717    2019-05-01        NaN    BSX   
196718      196718    2019-05-02        NaN    BSX   
...            ...           ...        ...    ...   
197160      197160    2024-06-28        NaN    BSX   
197161      197161    2024-07-01        NaN    BSX   
197162      197162    2024-07-15        NaN    BSX   
197163      197163    2024-07-16        NaN    BSX   
197164      197164    2024-07-17        NaN    BSX   

                                               Embeddings  
196714  [ 5.84030480e-01 -1.86310810e-01  6.35404867e-...  
196715  [ 4.34492740e-01 -3.38214015e-01  5.02259115e-...  
196716  [ 7.77825060e-01 -3.77962380e-01  7.86976800e-...  
196717  [ 0.676875   -0.2416388   0.5907422   0.411586...  
196718  [ 3.60225140e-01 -4.93427800e-01  4.5043510

<ipython-input-19-d1fbd9a6ecb2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['Compressed_Embeddings'] = compressed_embeddings.tolist()
<ipython-input-19-d1fbd9a6ecb2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['tic'] = final_df['symbol']
<ipython-input-19-d1fbd9a6ecb2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

        Unnamed: 0 publishedDate  embedding symbol  \
152252      152252    2016-01-13        NaN    ROK   
152253      152253    2016-04-13        NaN    ROK   
152254      152254    2016-07-13        NaN    ROK   
152255      152255    2016-10-21        NaN    ROK   
152256      152256    2017-01-12        NaN    ROK   
...            ...           ...        ...    ...   
152625      152625    2024-07-05        NaN    ROK   
152626      152626    2024-07-12        NaN    ROK   
152627      152627    2024-07-13        NaN    ROK   
152628      152628    2024-07-15        NaN    ROK   
152629      152629    2024-07-19        NaN    ROK   

                                               Embeddings  
152252  [-1.61300660e-01 -5.86403600e-01  4.23950200e-...  
152253  [ 1.53124030e-01 -4.85151800e-01  4.30608570e-...  
152254  [ 1.16105520e-02 -4.61703200e-01  7.44961800e-...  
152255  [-5.47400380e-02 -3.90017420e-01  7.57535040e-...  
152256  [-8.40246000e-02 -4.60271660e-01  4.8982100

...

<ipython-input-19-d1fbd9a6ecb2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['Compressed_Embeddings'] = compressed_embeddings.tolist()
<ipython-input-19-d1fbd9a6ecb2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['tic'] = final_df['symbol']


       Unnamed: 0 publishedDate  embedding symbol  \
36188       36188    2016-01-28        NaN    IRM   
36189       36189    2016-04-06        NaN    IRM   
36190       36190    2016-07-14        NaN    IRM   
36191       36191    2016-10-06        NaN    IRM   
36192       36192    2017-01-24        NaN    IRM   
...           ...           ...        ...    ...   
36672       36672    2024-06-28        NaN    IRM   
36673       36673    2024-07-03        NaN    IRM   
36674       36674    2024-07-09        NaN    IRM   
36675       36675    2024-07-15        NaN    IRM   
36676       36676    2024-07-18        NaN    IRM   

                                              Embeddings  
36188  [ 1.43788810e-01 -3.18971630e-01  4.68442440e-...  
36189  [ 1.73933760e-01 -4.34720550e-01  2.47003120e-...  
36190  [ 3.79395750e-01 -3.82529050e-01  6.00128170e-...  
36191  [ 2.40547760e-01 -3.37973950e-01  2.83930270e-...  
36192  [ 2.39397050e-01 -2.87694450e-01  3.62871400e-...  
...      

<ipython-input-19-d1fbd9a6ecb2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['Compressed_Embeddings'] = compressed_embeddings.tolist()
<ipython-input-19-d1fbd9a6ecb2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['tic'] = final_df['symbol']


       Unnamed: 0 publishedDate  embedding symbol  \
36677       36677    2016-03-01        NaN   DLTR   
36678       36678    2016-05-26        NaN   DLTR   
36679       36679    2016-08-25        NaN   DLTR   
36680       36680    2016-11-22        NaN   DLTR   
36681       36681    2017-03-01        NaN   DLTR   
...           ...           ...        ...    ...   
37262       37262    2024-06-29        NaN   DLTR   
37263       37263    2024-07-01        NaN   DLTR   
37264       37264    2024-07-02        NaN   DLTR   
37265       37265    2024-07-16        NaN   DLTR   
37266       37266    2024-07-18        NaN   DLTR   

                                              Embeddings  
36677  [ 0.21740045 -1.036377    0.7600505   0.098854...  
36678  [ 2.54864000e-01 -1.07682290e+00  2.57858280e-...  
36679  [ 4.96742760e-01 -1.02013350e+00  3.03049700e-...  
36680  [ 3.90181480e-01 -1.01627600e+00  5.41379800e-...  
36681  [ 3.61859500e-01 -9.45800800e-01  5.82103100e-...  
...      

<ipython-input-19-d1fbd9a6ecb2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['Compressed_Embeddings'] = compressed_embeddings.tolist()
<ipython-input-19-d1fbd9a6ecb2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['tic'] = final_df['symbol']


In [ ]:
compressed_news_embeddings.to_csv('/content/drive/MyDrive/compressed_news_embeddings.csv')

In [ ]:
compressed_news_embeddings = pd.read_csv('/content/drive/MyDrive/compressed_news_embeddings.csv')

In [ ]:
print(compressed_news_embeddings)

        Unnamed: 0.1  Unnamed: 0 publishedDate  embedding symbol  \
0                  0      179378    2016-10-17        NaN    STZ   
1                  1      179379    2019-09-25        NaN    STZ   
2                  2      179380    2019-10-04        NaN    STZ   
3                  3      179381    2019-12-03        NaN    STZ   
4                  4      179382    2019-12-09        NaN    STZ   
...              ...         ...           ...        ...    ...   
163900        163900       37262    2024-06-29        NaN   DLTR   
163901        163901       37263    2024-07-01        NaN   DLTR   
163902        163902       37264    2024-07-02        NaN   DLTR   
163903        163903       37265    2024-07-16        NaN   DLTR   
163904        163904       37266    2024-07-18        NaN   DLTR   

                                               Embeddings  \
0       [-0.10805868 -0.26979524  0.11729533  0.296641...   
1       [-2.30529790e-01  1.52359010e-01  7.03643800e-...   


In [ ]:
c

In [ ]:
compressed_news_embeddings = compressed_news_embeddings.drop(columns=['Embeddings','symbol'])

In [ ]:
compressed_news_embeddings.rename(columns={'Compressed_Embeddings': 'Embeddings'}, inplace=True)

In [ ]:
compressed_news_embeddings.rename(columns={'publishedDate': 'date'}, inplace=True)

In [ ]:
print((compressed_news_embeddings['tic']))

0          STZ
1          STZ
2          STZ
3          STZ
4          STZ
          ... 
163900    DLTR
163901    DLTR
163902    DLTR
163903    DLTR
163904    DLTR
Name: tic, Length: 163905, dtype: object


In [ ]:
merged_df = pd.merge(filtered_df, compressed_news_embeddings, on=['tic', 'date'], how='left')

# Step 2: Update the embeddings in DataFrame A with non-zero embeddings from DataFrame B
# If embeddings in df_B are not NaN, overwrite the corresponding embeddings in df_A
# merged_df['Embeddings_A'] = merged_df.apply(
#     lambda row: row['Embeddings_B'] if isinstance(row['Embeddings_B'], np.ndarray) else row['Embeddings_A'], axis=1
# )

# Step 3: Rename and drop unnecessary columns
# merged_df.rename(columns={'Embeddings_A': 'Embeddings'}, inplace=True)
# final_df = merged_df.drop(columns=['Embeddings_B'])

# Display the final DataFrame


In [ ]:
print(merged_df)

        Unnamed: 0.1_x  Unnamed: 0_x        date        open        high  \
0                    0             0  2016-01-04  136.030029  138.060457   
1                    1             1  2016-01-05  137.703265  139.254290   
2                    2             2  2016-01-06  134.855021  136.180436   
3                    3             3  2016-01-07  132.542533  134.384975   
4                    4             4  2016-01-08  133.482544  133.604751   
...                ...           ...         ...         ...         ...   
421024          421024        421024  2024-03-22  123.862256  124.140574   
421025          421025        421025  2024-03-25  123.136631  123.484537   
421026          421026        421026  2024-03-26  122.301669  123.116748   
421027          421027        421027  2024-03-27  123.226092  123.921889   
421028          421028        421028  2024-03-28  124.249916  124.746915   

               low       close   volume  dividends  stock splits  ...  \
0       135.44

In [ ]:
merged_df = pd.read_csv('/content/drive/MyDrive/final_df_compressed_merged_embeddings.csv')



In [ ]:
merged_df = merged_df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0', 'Unnamed: 0.1_x', 'Embeddings_x', 'Unnamed: 0.1_y', 'Unnamed: 0_y','embedding'])

In [ ]:
merged_df.rename(columns={'Embeddings_y': 'Embeddings'}, inplace=True)

In [ ]:
print(merged_df)

        Unnamed: 0        date        open        high         low  \
0                0  2016-01-04  136.030029  138.060457  135.447208   
1                1  2016-01-05  137.703265  139.254290  135.099411   
2                2  2016-01-06  134.855021  136.180436  134.187602   
3                3  2016-01-07  132.542533  134.384975  131.818715   
4                4  2016-01-08  133.482544  133.604751  130.267685   
...            ...         ...         ...         ...         ...   
421024      421024  2024-03-22  123.862256  124.140574  122.947778   
421025      421025  2024-03-25  123.136631  123.484537  122.331494   
421026      421026  2024-03-26  122.301669  123.116748  121.655568   
421027      421027  2024-03-27  123.226092  123.921889  122.470651   
421028      421028  2024-03-28  124.249916  124.746915  122.808620   

             close   volume  dividends  stock splits   day   tic  adj close  \
0       137.759659  1461700        0.0           0.0     0    CI        NaN   
1

In [ ]:
zero_vector = np.zeros(10)

# Step 2: Replace NaN embeddings with the zero vector
merged_df['Embeddings'] = merged_df['Embeddings'].apply(lambda x: zero_vector if pd.isna(x) else x)

In [ ]:
print(final_df[final_df['tic'] == 'STZ'])


        Unnamed: 0.1_A  Unnamed: 0_A        date        open        high  \
167913          167913        167913  2016-01-04  125.257638  125.302049   
167914          167914        167914  2016-01-05  125.728354  127.851064   
167915          167915        167915  2016-01-06  126.998409  128.259599   
167916          167916        167916  2016-01-07  132.780338  137.052399   
167917          167917        167917  2016-01-08  133.117912  134.112651   
...                ...           ...         ...         ...         ...   
169981          169981        169981  2024-03-22  268.786724  268.786724   
169982          169982        169982  2024-03-25  266.664180  268.399887   
169983          169983        169983  2024-03-26  268.856160  268.856160   
169984          169984        169984  2024-03-27  268.360272  269.957109   
169985          169985        169985  2024-03-28  270.155459  270.770390   

               low       close   volume  dividends  stock splits  ...  \
167913  122.96

In [ ]:
merged_df.to_csv('/content/drive/MyDrive/final_df_compressed_merged_embeddings.csv')

In [ ]:
print(merged_df)

        Unnamed: 0  Unnamed: 0.1_x  Unnamed: 0_x        date        open  \
0                0               0             0  2016-01-04  136.030029   
1                1               1             1  2016-01-05  137.703265   
2                2               2             2  2016-01-06  134.855021   
3                3               3             3  2016-01-07  132.542533   
4                4               4             4  2016-01-08  133.482544   
...            ...             ...           ...         ...         ...   
421024      421024          421024        421024  2024-03-22  123.862256   
421025      421025          421025        421025  2024-03-25  123.136631   
421026      421026          421026        421026  2024-03-26  122.301669   
421027      421027          421027        421027  2024-03-27  123.226092   
421028      421028          421028        421028  2024-03-28  124.249916   

              high         low       close   volume  dividends  ...  \
0       138.0604

In [ ]:
embeddings = np.stack(final_df['Embeddings'].values)

# Step 2: Apply PCA to reduce dimensions
pca = PCA(n_components=10)  # Specify the number of components you want (e.g., 10)
reduced_embeddings = pca.fit_transform(embeddings)

# Step 3: Convert the reduced embeddings to a DataFrame
reduced_embeddings_df = pd.DataFrame(reduced_embeddings, columns=[f'PC{i+1}' for i in range(reduced_embeddings.shape[1])])

# Step 4: Concatenate the reduced embeddings with the original DataFrame
final_df = pd.concat([final_df, reduced_embeddings_df], axis=1)

# Display the updated DataFrame
print(final_df)

In [ ]:
final_df.to_csv('/content/drive/MyDrive/computed_embeddings.csv')